In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
train_tape = "Z:/rppg/scamps_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [21]:
phys_net = PhysNet()
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Flops per frame:54260867
Model: "phys_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_33 (Sequential)   (None, 128, 32, 32, 16)   1280      
_________________________________________________________________
sequential_34 (Sequential)   (None, 128, 16, 16, 32)   13984     
_________________________________________________________________
sequential_35 (Sequential)   (None, 128, 16, 16, 64)   55616     
_________________________________________________________________
sequential_36 (Sequential)   (None, 64, 8, 8, 64)      110912    
_________________________________________________________________
sequential_37 (Sequential)   (None, 64, 8, 8, 64)      110912    
_________________________________________________________________
sequential_38 (Sequential)   (None, 32, 4, 4, 64)      110912    
_________________________________________________________________
sequential_39 (Sequential)   (N

In [22]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_SCAMPS.h5')

Epoch 1/3
374/374 [==============================] - 133s 336ms/step - loss: 0.9654 - val_loss: 0.9439

Epoch 00001: val_loss improved from inf to 0.94386, saving model to ../weights\PhysNet_SCAMPS.h5
Epoch 2/3
374/374 [==============================] - 132s 336ms/step - loss: 0.6449 - val_loss: 0.3890

Epoch 00002: val_loss improved from 0.94386 to 0.38898, saving model to ../weights\PhysNet_SCAMPS.h5
Epoch 3/3
374/374 [==============================] - 132s 337ms/step - loss: 0.3546 - val_loss: 0.2978

Epoch 00003: val_loss improved from 0.38898 to 0.29776, saving model to ../weights\PhysNet_SCAMPS.h5


In [7]:
phys_net.load_weights('../weights/PhysNet_SCAMPS.h5')

In [23]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:18<00:00,  3.24it/s]


{'Sliding window': {'MAE': 31.787, 'RMSE': 40.093, 'R': 0.18123},
 'Whole video': {'MAE': 28.616, 'RMSE': 37.426, 'R': 0.20362}}

In [24]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:12<00:00,  3.44it/s]


{'Sliding window': {'MAE': 5.707, 'RMSE': 11.274, 'R': 0.76438},
 'Whole video': {'MAE': 4.489, 'RMSE': 8.521, 'R': 0.87463}}